# Bicycle data melbourne analysis

In [49]:
import pandas as pd
import numpy as np
import os
from collections import Counter
from tqdm.notebook import tqdm

## Data validation 

The bicycle data is delivered in a large number of CSV files, without an accompanying data dictionary.  To be sure the same columns are shared by all CSVs, such that they can be appended to a master file, we iterate over them and tally up the unique column combinations.  Ideally, the results will be a single combination of columns with a tally count the length of the number of CSV files.  Let's see!

In [ ]:
rootdir ='../data'
tally = pd.DataFrame(columns=["count","len"])
counter = 0
for subdir, dirs, files in tqdm(os.walk(rootdir)):
    for file in files:
        if ('.csv' in file) and ('.zip' not in file):
            # read in CSV, if it contains records (which at least one doesn't!)
            file_path = os.path.join(subdir,file)
            if os.path.getsize(file_path) > 0:
                df = pd.read_csv(os.path.join(subdir,file))
                # store list of columns in variable df_columns as a string
                df_columns = f"{df.columns.to_list()}"
                # if CSV columns string is in the tally index, increment this
                if df_columns in tally.index:
                    tally[tally.index==df_columns] += 1
                # otherwise add CSV columns string to the tally index
                else:
                    tally.loc[df_columns] = 1
                # increment a counter; athough theoretically this should only sum to the sum of tallys!
                counter+=1

print(counter)

In [8]:
tally

,count
"['DATA_TYPE', 'TIS_DATA_REQUEST', 'SITE_XN_ROUTE', 'LOC_LEG', 'DATE', 'TIME', 'CLASS', 'LANE', 'SPEED', 'WHEELBASE', 'HEADWAY', 'GAP', 'AXLE', 'AXLE_GROUPING', 'RHO', 'VEHICLE', 'DIRECTION']",13229


In [12]:
test = pd.read_csv(os.path.join(subdir,file))

## Creating a master dataframe

Now that we know that all of the CSVs share the same column names, we will join the various files together to create a master dataframe to run the analysis on. 

In [ ]:
rootdir ='../data'
csv_files = []

for subdir, dirs, files in tqdm(os.walk(rootdir)):
    for file in files:
        if ('.csv' in file) and ('.zip' not in file):
            # record filepaths of CSVs containing records
            file_path = os.path.join(subdir,file)
            if os.path.getsize(file_path) > 0:
                csv_files.append(os.path.join(subdir,file))

df = pd.concat([pd.read_csv(file) for file in tqdm(csv_files,desc="Joining csv files...",unit="files")])            
df.info()


0it [00:00, ?it/s]

Joining csv files...:   0%|          | 0/16054 [00:00<?, ?files/s]

In [31]:
df.to_csv('master_dataframe.csv')

## Summary statistics

Let's look at the summary statistics for the master dataframe! First we'll look at what type of data is included in each column. Then we can see can check the max/min values, and the distribution of the data, to see if there are any outlying data points. 

In [32]:
#Checking data types in each column

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4925 entries, 0 to 4924
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DATA_TYPE         4925 non-null   object 
 1   TIS_DATA_REQUEST  4925 non-null   int64  
 2   SITE_XN_ROUTE     4925 non-null   int64  
 3   LOC_LEG           4925 non-null   int64  
 4   DATE              4925 non-null   object 
 5   TIME              4925 non-null   object 
 6   CLASS             4925 non-null   int64  
 7   LANE              4925 non-null   int64  
 8   SPEED             4925 non-null   float64
 9   WHEELBASE         4925 non-null   float64
 10  HEADWAY           4925 non-null   float64
 11  GAP               4925 non-null   float64
 12  AXLE              4925 non-null   int64  
 13  AXLE_GROUPING     4925 non-null   int64  
 14  RHO               4925 non-null   float64
 15  VEHICLE           4925 non-null   object 
 16  DIRECTION         4925 non-null   object 


In [34]:
#Summary statistics

df.describe()

,TIS_DATA_REQUEST,SITE_XN_ROUTE,LOC_LEG,CLASS,LANE,SPEED,WHEELBASE,HEADWAY,GAP,AXLE,AXLE_GROUPING,RHO
count,4925.0,4925.0,4925.000000,4925.0,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.0,4925.0,4925.000000
mean,208.0,10223.0,59443.482437,15.0,0.482437,18.764264,1.001401,473.021401,472.778518,2.0,1.0,0.984083
std,0.0,0.0,0.499742,0.0,0.499742,7.449773,0.142950,2259.648522,2259.641277,0.0,0.0,0.081180
min,208.0,10223.0,59443.000000,15.0,0.000000,1.400000,0.000000,0.000000,0.000000,2.0,1.0,0.200000
25%,208.0,10223.0,59443.000000,15.0,0.000000,13.200000,1.000000,5.600000,5.400000,2.0,1.0,1.000000
50%,208.0,10223.0,59443.000000,15.0,0.000000,18.500000,1.000000,109.100000,108.700000,2.0,1.0,1.000000
75%,208.0,10223.0,59444.000000,15.0,1.000000,23.900000,1.100000,362.800000,361.700000,2.0,1.0,1.000000
max,208.0,10223.0,59444.000000,15.0,1.000000,120.500000,4.000000,35194.600000,35194.400000,2.0,1.0,1.200000


In [44]:
os.path.getsize('../data\Bicycle_Volume_Speed_2020\Black Rock - Bicycle volume data 2020\IND_D5555_X32021.csv.20201228')

0